# Wildfire Data Preparation and Subsetting for Analysis


### Subsetting Criteria

The following criteria are applied to the data to narrow the scope:

1. **Temporal Range**: The data is limited to the last 60 years, specifically between **1961 and 2021**. This timeframe provides a more recent view of wildfire activity and is more relevant for studying modern fire trends.
   
2. **Proximity to Assigned City**: Only fires occurring within **650 miles of the assigned city (Tallahassee, Fl)** are included. This spatial filter allows us to focus on fires that have a direct geographical impact on the region of interest.

3. **Seasonal Filter**: The annual wildfire season is defined as running from **May 1st through October 31st**. Fires within this period are more likely to be impactful in terms of seasonal trends, resource allocation, and environmental effects.

### Resources

The complete [Wildfire dataset](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) can be retrieved from a US government repository.

This notebook has dependencies on [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html), the [geojson](https://pypi.org/project/geojson/) module, and the wildfire user module. Pyproj and geojson can be installed via pip.

### License
This notebook also relies on code that was developed from code provided by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - August 16, 2024.

### Preliminaries
First we start with some imports and some constant definitions.

In [1]:
import re
import os
import pandas as pd
from datetime import datetime

In [2]:
# local imports
# from Reader import Reader as WFReader
from Wildfire_JSON_Reader import load_wildfire_features
import Geo_Calc as GC

### Lets subset by dates first
The geojson files are large, but we also have a csv that contains fire information including dates. We will first read in this data and do some of the date subsetting beforehand to make the process go faster. 

Read in the fire data

In [3]:
widfire_csv = 'USGS_Wildland_Fire_Combined_Dataset.csv'
wildfire_csv_path = r'C:\Users\clark.roll\python coding\code_personal\Data512_ProjectDataset'
csv_filepath = os.path.join(wildfire_csv_path, widfire_csv)
df = pd.read_csv(csv_filepath)

C:\Users\clark.roll\AppData\Local\Temp\ipykernel_32592\3079598232.py:4: DtypeWarning: Columns (0,1,2,3,5,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_filepath)


subset to just the columns we need for date and to track specific fires

In [4]:
# Define a function to extract different date types from 'Listed_Fire_Dates'
def extract_dates(listed_dates):
    if pd.isna(listed_dates):
        return {}
    date_matches = re.findall(r'(\w+ \w+ Date\(s\)): (\d{4}-\d{2}-\d{2})', listed_dates)
    return {match[0]: match[1] for match in date_matches}

In [5]:
# Apply the function to the 'Listed_Fire_Dates' column and create new columns based on the extracted dates
dates_extracted = df['Listed_Fire_Dates'].apply(extract_dates)
dates_df = pd.DataFrame(dates_extracted.tolist())

# Concatenate the original subset DataFrame with the new extracted date columns
dates_df = pd.concat([df, dates_df], axis=1)

Clean up a couple of columns to ensure they're the right data types

In [6]:
# Convert the 'Fire_Year' column to numeric, coercing errors, and drop rows where conversion fails
dates_df['Fire_Year'] = pd.to_numeric(dates_df['Fire_Year'], errors='coerce')

# Drop rows with NaN values in the 'Fire_Year' column after conversion
dates_df = dates_df.dropna(subset=['Fire_Year'])

# Convert the 'Fire_Year' column to integer type
dates_df['Fire_Year'] = dates_df['Fire_Year'].astype(int)

### Subset by years
Lets select just the data that is within the years that we need

In [7]:
# Create a copy of the DataFrame to avoid SettingWithCopyWarning
dates_df = dates_df.copy()

# Define the year range for filtering fires
start_year = 1961
end_year = 2021

# Filter the DataFrame to include only rows where 'Fire_Year' is within the desired range
filtered_df = dates_df[(dates_df['Fire_Year'] >= start_year) & (dates_df['Fire_Year'] <= end_year)].copy()

Lets look at the output

In [8]:
filtered_df

,OID_,USGS_Assigned_ID,Assigned_Fire_Type,Fire_Year,Fire_Polygon_Tier,Fire_Attribute_Tiers,GIS_Acres,GIS_Hectares,Source_Datasets,Listed_Fire_Types,...,Shape_Area,Other Fire Date(s),Listed Upload Date(s),Wildfire Discovery Date(s),Wildfire Controlled Date(s),Wildfire Out Date(s),Wildfire Containment Date(s),Fire Start Date(s),Fire End Date(s),Listed Ignition Date(s)
13525,13526,13526,Wildfire,1961,1.0,"1 (4), 3 (6)",55249.740141,22358.776573,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (8), Likely Wildfire (2)",...,2.235878e+08,1899-12-30,NaN,1961-09-08,NaN,NaN,NaN,NaN,NaN,NaN
13526,13527,13527,Wildfire,1961,1.0,"1 (4), 3 (6)",43920.841077,17774.133779,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (8), Likely Wildfire (2)",...,1.777413e+08,1899-12-30,NaN,1961-07-10,NaN,NaN,NaN,NaN,NaN,NaN
13527,13528,13528,Wildfire,1961,1.0,"1 (4), 3 (6)",34233.698603,13853.886306,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (8), Likely Wildfire (2)",...,1.385389e+08,1899-12-30,NaN,1961-09-02,NaN,NaN,NaN,NaN,NaN,NaN
13528,13529,13529,Wildfire,1961,1.0,"1 (2), 3 (3)",32866.053451,13300.419949,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (4), Likely Wildfire (1)",...,1.330042e+08,1961-07-12,NaN,1961-07-12,NaN,NaN,NaN,NaN,NaN,NaN
13529,13530,13530,Wildfire,1961,1.0,"1 (1), 3 (2)",27269.045936,11035.391368,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (2), Likely Wildfire (1)",...,1.103539e+08,2017-12-18,NaN,1961-09-01,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444174,135057,135057.0,Prescribed Fire,2020,8.0,8 (3),16.412148,6.641761,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (3),...,6.641761e+04,NaN,2020-09-11,NaN,NaN,NaN,NaN,NaN,2020-01-01,NaN
444175,135058,135058.0,Prescribed Fire,2020,8.0,8 (1),7.050837,2.853373,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (1),...,2.853373e+04,NaN,2021-01-08,NaN,NaN,NaN,NaN,NaN,2020-05-16,NaN
444176,135059,135059.0,Prescribed Fire,2020,8.0,8 (4),9.342668,3.780843,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (4),...,3.780843e+04,NaN,2021-01-08,NaN,NaN,NaN,NaN,NaN,2020-05-16,NaN
444177,135060,135060.0,Prescribed Fire,2020,8.0,8 (1),0.996962,0.403456,Comb_National_Rx_Only_BLM_VTRT_Prescribed_Fire...,Prescribed Fire (1),...,4.034562e+03,NaN,2021-02-05,NaN,NaN,NaN,NaN,2020-07-06,NaN,NaN


### Subset by Months
Theres lots of nans in our dataset and we have multiple possible start date columns. We are just interested in fires that are started during the regular fire season so March - October 

with multiple columns recording start date we want to get the most trusted start date- so we will create a function that will get the start date columns and select the date where the year matches the fire year to ensure internal consistency 

In [9]:
# Create a function to extract a trusted start date from a row

def get_trusted_start_date(row: pd.Series) -> datetime:
    """
    Extract a trusted start date from a row by checking specific
    start date columns that match the fire year.

    Parameters
    ----------
    row : pd.Series
        A row of the DataFrame containing fire information,
        including multiple start date columns.

    Returns
    -------
    datetime or pd.NaT
        The trusted start date that matches the fire year, or
        pd.NaT if no matching date is found.
    """
    fire_year = row['Fire_Year']
    start_date_columns = [
        "Wildfire Discovery Date(s)",
        "Fire Start Date(s)",
        "Listed Ignition Date(s)"
    ]
    
    for col in start_date_columns:
        date_str = row.get(col)
        
        if pd.notna(date_str):
            try:
                # Use datetime.strptime to parse the date string
                date_obj = datetime.strptime(date_str, "%Y-%m-%d")
                if date_obj.year == fire_year:
                    return date_obj
            except ValueError:
                continue
    return pd.NaT

In [10]:
# Apply the function to each row to create the 'Trusted_Date' column
filtered_df['Trusted_Start_Date'] = filtered_df.apply(get_trusted_start_date, axis=1)

We now have to remove rows where we don't have a trusted start date because we can't know if it is within our May-October timeframe

In [11]:
# Filter out rows without a trusted start date
filtered_df = filtered_df.dropna(subset=['Trusted_Start_Date'])

# Filter rows where 'Trusted_Start_Date' is between May and October
filtered_df = filtered_df[(filtered_df['Trusted_Start_Date'].dt.month >= 5) &
                          (filtered_df['Trusted_Start_Date'].dt.month <= 10)]

# Ensure 'USGS_Assigned_ID' is an integer
filtered_df['USGS_Assigned_ID'] = filtered_df['USGS_Assigned_ID'].astype(int)

Lets get the fire ids for our subset of of data

In [12]:
# Create a set of USGS_Assigned_IDs from the filtered DataFrame
usgs_fire_ids = set(filtered_df['USGS_Assigned_ID'])

## Calculate fields & Subset Metadata

while we are here and have most of this work done we are going to calculate just a couple more fields and save a subset of the metadata

In [13]:
# Create a function to extract a trusted end date from a row
def get_trusted_end_date(row: pd.Series) -> datetime:
    """
    Extract a trusted end date from a row by checking specific
    end date columns that match the fire year.

    Parameters
    ----------
    row : pd.Series
        A row of the DataFrame containing fire information,
        including multiple end date columns.

    Returns
    -------
    datetime
        The trusted end date that matches the fire year, or
        October 31 of the fire year if no matching date is found.
        Returns NaN if the Trusted Start Date is after the end date.
    """
    # Ensure fire_year is a valid integer; return NaT if missing
    fire_year = row['Fire_Year']
    if pd.isna(fire_year):
        return pd.NaT
    fire_year = int(fire_year)

    end_date_columns = [
        "Wildfire Out Date(s)",
        "Fire End Date(s)"
    ]
    
    for col in end_date_columns:
        date_str = row.get(col)
        
        if pd.notna(date_str):
            try:
                # Use datetime.strptime to parse the date string
                date_obj = datetime.strptime(date_str, "%Y-%m-%d")
                if date_obj.year == fire_year:
                    # Check if Trusted_Start_Date is before this end date
                    trusted_start_date = row.get('Trusted_Start_Date')
                    if pd.notna(trusted_start_date):
                        # Parse Trusted_Start_Date if it's a string, otherwise assume it's datetime
                        if isinstance(trusted_start_date, str):
                            trusted_start_date = datetime.strptime(trusted_start_date, "%Y-%m-%d")
                        # Return NaT if Trusted_Start_Date is after the end date
                        if trusted_start_date > date_obj:
                            return pd.NaT
                    return date_obj
            except ValueError:
                continue
    
    # Default to October 31 of the fire year if no matching date is found
    default_end_date = datetime(fire_year, 10, 31)
    trusted_start_date = row.get('Trusted_Start_Date')
    if pd.notna(trusted_start_date):
        if isinstance(trusted_start_date, str):
            trusted_start_date = datetime.strptime(trusted_start_date, "%Y-%m-%d")
        # Return NaT if Trusted_Start_Date is after the default end date
        if trusted_start_date > default_end_date:
            return pd.NaT
    
    return default_end_date

In [14]:
# Apply the function to each row to create the 'Trusted_Date' column
filtered_df['Trusted_End_Date'] = filtered_df.apply(get_trusted_end_date, axis=1)

# Remove the rows where  Trusted End Date is NAN
filtered_df.dropna(subset=['Trusted_End_Date'], inplace = True)

# Now we can get fire duration! Huzzah!
filtered_df['duration_days'] = (filtered_df['Trusted_End_Date'] - filtered_df['Trusted_Start_Date']).dt.days

In [15]:
filtered_df.columns

Index(['OID_', 'USGS_Assigned_ID', 'Assigned_Fire_Type', 'Fire_Year',
       'Fire_Polygon_Tier', 'Fire_Attribute_Tiers', 'GIS_Acres',
       'GIS_Hectares', 'Source_Datasets', 'Listed_Fire_Types',
       'Listed_Fire_Names', 'Listed_Fire_Codes', 'Listed_Fire_IDs',
       'Listed_Fire_IRWIN_IDs', 'Listed_Fire_Dates', 'Listed_Fire_Causes',
       'Listed_Fire_Cause_Class', 'Listed_Rx_Reported_Acres',
       'Listed_Map_Digitize_Methods', 'Listed_Notes', 'Processing_Notes',
       'Wildfire_Notice', 'Prescribed_Burn_Notice', 'Wildfire_and_Rx_Flag',
       'Overlap_Within_1_or_2_Flag', 'Circleness_Scale', 'Circle_Flag',
       'Exclude_From_Summary_Rasters', 'Shape_Length', 'Shape_Area',
       'Other Fire Date(s)', 'Listed Upload Date(s)',
       'Wildfire Discovery Date(s)', 'Wildfire Controlled Date(s)',
       'Wildfire Out Date(s)', 'Wildfire Containment Date(s)',
       'Fire Start Date(s)', 'Fire End Date(s)', 'Listed Ignition Date(s)',
       'Trusted_Start_Date', 'Trusted_End_Dat

Now lets subset this to just the columns we're going to use in our smoke model and for data visualization and save it!

In [16]:
final_df = filtered_df[['USGS_Assigned_ID', 'Assigned_Fire_Type',
                        'Fire_Year','Trusted_Start_Date',
                        'Trusted_End_Date', 'duration_days','GIS_Acres']].copy()

final_df.to_csv('Wildland_Fire_Combined_Subset.csv', index = False)
final_df

,USGS_Assigned_ID,Assigned_Fire_Type,Fire_Year,Trusted_Start_Date,Trusted_End_Date,duration_days,GIS_Acres
13525,13526,Wildfire,1961,1961-09-08,1961-10-31,53,55249.740141
13526,13527,Wildfire,1961,1961-07-10,1961-10-31,113,43920.841077
13527,13528,Wildfire,1961,1961-09-02,1961-10-31,59,34233.698603
13528,13529,Wildfire,1961,1961-07-12,1961-10-31,111,32866.053451
13529,13530,Wildfire,1961,1961-09-01,1961-10-31,60,27269.045936
...,...,...,...,...,...,...,...
444127,135010,Prescribed Fire,2020,2020-05-01,2020-07-30,90,4143.012270
444134,135017,Prescribed Fire,2020,2020-08-12,2020-08-20,8,206.093386
444145,135028,Prescribed Fire,2020,2020-05-01,2020-05-02,1,77.245054
444164,135047,Prescribed Fire,2020,2020-06-20,2020-10-31,133,19.824395


### Load the json data

In [15]:
wildfire_json = 'USGS_Wildland_Fire_Combined_Dataset.json'
wildfire_data_directory = r'C:\Users\kater\AI_python_packages\data512_dataset\GeoJSON Files\GeoJSON Exports'
json_file_path = os.path.join(wildfire_data_directory, wildfire_json)

In [16]:
wildfire_data =  load_wildfire_features(json_file_path, 
                                        max_features=136000)

Attempting to open 'C:\Users\kater\AI_python_packages\data512_dataset\GeoJSON Files\GeoJSON Exports\USGS_Wildland_Fire_Combined_Dataset.json' with wildfire.Reader() object

The header contains the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields']
Header Dictionary:
{
    "displayFieldName": "",
    "fieldAliases": {
        "OBJECTID": "OBJECTID",
        "USGS_Assigned_ID": "USGS Assigned ID",
        "Assigned_Fire_Type": "Assigned Fire Type",
        "Fire_Year": "Fire Year",
        "Fire_Polygon_Tier": "Fire Polygon Tier",
        "Fire_Attribute_Tiers": "Fire Attribute Tiers",
        "GIS_Acres": "GIS_Acres",
        "GIS_Hectares": "GIS_Hectares",
        "Source_Datasets": "Source Datasets",
        "Listed_Fire_Types": "Listed Fire Types",
        "Listed_Fire_Names": "Listed Fire Names",
        "Listed_Fire_Codes": "Listed Fire Codes",
        "Listed_Fire_IDs": "Listed Fire IDs",
        "Listed_Fire_IRWIN_IDs": "Listed Fire

Loaded 5900 features
Loaded 6000 features
Loaded 6100 features
Loaded 6200 features
Loaded 6300 features
Loaded 6400 features
Loaded 6500 features
Loaded 6600 features
Loaded 6700 features
Loaded 6800 features
Loaded 6900 features
Loaded 7000 features
Loaded 7100 features
Loaded 7200 features
Loaded 7300 features
Loaded 7400 features
Loaded 7500 features
Loaded 7600 features
Loaded 7700 features
Loaded 7800 features
Loaded 7900 features
Loaded 8000 features
Loaded 8100 features
Loaded 8200 features
Loaded 8300 features
Loaded 8400 features
Loaded 8500 features
Loaded 8600 features
Loaded 8700 features
Loaded 8800 features
Loaded 8900 features
Loaded 9000 features
Loaded 9100 features
Loaded 9200 features
Loaded 9300 features
Loaded 9400 features
Loaded 9500 features
Loaded 9600 features
Loaded 9700 features
Loaded 9800 features
Loaded 9900 features
Loaded 10000 features
Loaded 10100 features
Loaded 10200 features
Loaded 10300 features
Loaded 10400 features
Loaded 10500 features
Loaded 

Loaded 43400 features
Loaded 43500 features
Loaded 43600 features
Loaded 43700 features
Loaded 43800 features
Loaded 43900 features
Loaded 44000 features
Loaded 44100 features
Loaded 44200 features
Loaded 44300 features
Loaded 44400 features
Loaded 44500 features
Loaded 44600 features
Loaded 44700 features
Loaded 44800 features
Loaded 44900 features
Loaded 45000 features
Loaded 45100 features
Loaded 45200 features
Loaded 45300 features
Loaded 45400 features
Loaded 45500 features
Loaded 45600 features
Loaded 45700 features
Loaded 45800 features
Loaded 45900 features
Loaded 46000 features
Loaded 46100 features
Loaded 46200 features
Loaded 46300 features
Loaded 46400 features
Loaded 46500 features
Loaded 46600 features
Loaded 46700 features
Loaded 46800 features
Loaded 46900 features
Loaded 47000 features
Loaded 47100 features
Loaded 47200 features
Loaded 47300 features
Loaded 47400 features
Loaded 47500 features
Loaded 47600 features
Loaded 47700 features
Loaded 47800 features
Loaded 479

Loaded 80700 features
Loaded 80800 features
Loaded 80900 features
Loaded 81000 features
Loaded 81100 features
Loaded 81200 features
Loaded 81300 features
Loaded 81400 features
Loaded 81500 features
Loaded 81600 features
Loaded 81700 features
Loaded 81800 features
Loaded 81900 features
Loaded 82000 features
Loaded 82100 features
Loaded 82200 features
Loaded 82300 features
Loaded 82400 features
Loaded 82500 features
Loaded 82600 features
Loaded 82700 features
Loaded 82800 features
Loaded 82900 features
Loaded 83000 features
Loaded 83100 features
Loaded 83200 features
Loaded 83300 features
Loaded 83400 features
Loaded 83500 features
Loaded 83600 features
Loaded 83700 features
Loaded 83800 features
Loaded 83900 features
Loaded 84000 features
Loaded 84100 features
Loaded 84200 features
Loaded 84300 features
Loaded 84400 features
Loaded 84500 features
Loaded 84600 features
Loaded 84700 features
Loaded 84800 features
Loaded 84900 features
Loaded 85000 features
Loaded 85100 features
Loaded 852

Loaded 117200 features
Loaded 117300 features
Loaded 117400 features
Loaded 117500 features
Loaded 117600 features
Loaded 117700 features
Loaded 117800 features
Loaded 117900 features
Loaded 118000 features
Loaded 118100 features
Loaded 118200 features
Loaded 118300 features
Loaded 118400 features
Loaded 118500 features
Loaded 118600 features
Loaded 118700 features
Loaded 118800 features
Loaded 118900 features
Loaded 119000 features
Loaded 119100 features
Loaded 119200 features
Loaded 119300 features
Loaded 119400 features
Loaded 119500 features
Loaded 119600 features
Loaded 119700 features
Loaded 119800 features
Loaded 119900 features
Loaded 120000 features
Loaded 120100 features
Loaded 120200 features
Loaded 120300 features
Loaded 120400 features
Loaded 120500 features
Loaded 120600 features
Loaded 120700 features
Loaded 120800 features
Loaded 120900 features
Loaded 121000 features
Loaded 121100 features
Loaded 121200 features
Loaded 121300 features
Loaded 121400 features
Loaded 1215

Filter the data to just the data that was within our time range so we have less fires to calculate the distances for

In [17]:
# Filter the wildfire features list to keep only features with a matching USGS_Assigned_ID
wildfire_features_filtered = [
    feature for feature in wildfire_data
    if feature['attributes'].get('USGS_Assigned_ID') in usgs_fire_ids
]

## Calculate distances from our city (Tallahassee) to the fire perimeter

In [22]:
TALAHASSEE_COORDS = [30.46, -84.25] 

fire_distances = GC.process_fire_distances_from_location(TALAHASSEE_COORDS,
                                                         wildfire_features_filtered,
                                                         verbose = True)

Processed 100 features
Processed 200 features
Processed 300 features
Processed 400 features
Processed 500 features
Processed 600 features
Processed 700 features
Processed 800 features
Processed 900 features
Processed 1000 features
Processed 1100 features
Processed 1200 features
Processed 1300 features
Processed 1400 features
Processed 1500 features
Processed 1600 features
Processed 1700 features
Processed 1800 features
Processed 1900 features
Processed 2000 features
Processed 2100 features
Processed 2200 features
Processed 2300 features
Processed 2400 features
Processed 2500 features
Processed 2600 features
Processed 2700 features
Processed 2800 features
Processed 2900 features
Processed 3000 features
Processed 3100 features
Processed 3200 features
Processed 3300 features
Processed 3400 features
Processed 3500 features
Processed 3600 features
Processed 3700 features
Processed 3800 features
Processed 3900 features
Processed 4000 features
Processed 4100 features
Processed 4200 features
P

Processed 33400 features
Processed 33500 features
Processed 33600 features
Processed 33700 features
Processed 33800 features
Processed 33900 features
Processed 34000 features
Processed 34100 features
Processed 34200 features
Processed 34300 features
Processed 34400 features
Processed 34500 features
Processed 34600 features
Processed 34700 features
Processed 34800 features
Processed 34900 features
Processed 35000 features
Processed 35100 features
Processed 35200 features
Processed 35300 features
Processed 35400 features
Processed 35500 features
Processed 35600 features
Processed 35700 features
Processed 35800 features
Processed 35900 features
Processed 36000 features
Processed 36100 features
Processed 36200 features
Processed 36300 features
Processed 36400 features
Processed 36500 features
Processed 36600 features
Processed 36700 features
Processed 36800 features
Processed 36900 features
Processed 37000 features
Processed 37100 features
Processed 37200 features
Processed 37300 features


Okay lets look at the data we gathered!!

In [23]:
fire_distances

,usgs_assigned_id,fire_year,fire_dates,fire_name,fire_size_acres,fire_type,closest_distance_miles,closest_point_lat,closest_point_lon,average_distance_miles
0,13526,1961,Listed Wildfire Discovery Date(s): 1961-09-08 ...,BIG ANTELOPE (5),55249.740141,Wildfire,2212.293403,40.310411,-121.895451,2218.332334
1,13527,1961,Listed Wildfire Discovery Date(s): 1961-07-10 ...,HARLOW (5),43920.841077,Wildfire,2077.348836,37.268174,-119.675518,2078.078157
2,13528,1961,Listed Wildfire Discovery Date(s): 1961-09-02 ...,RANCHERIA (5),34233.698603,Wildfire,2135.116430,38.447980,-120.652318,2142.401009
3,13529,1961,Listed Wildfire Discovery Date(s): 1961-07-12 ...,BOLLINGER RIDGE (5),32866.053451,Wildfire,2169.219255,37.233625,-121.344383,2175.600421
4,13530,1961,Listed Wildfire Discovery Date(s): 1961-09-01 ...,Ditch Creek (3),27269.045936,Wildfire,2139.278205,44.922865,-119.253744,2142.886328
...,...,...,...,...,...,...,...,...,...,...
53925,135010,2020,Listed Prescribed Fire Start Date(s): 2020-05-...,2_FIAT_BigDesertPPA_HL_MA_USFONoxWeeds_FB41_Ch...,4143.012270,Prescribed Fire,1783.951846,44.448183,-111.836191,1784.887371
53926,135017,2020,Listed Prescribed Fire Start Date(s): 2020-08-...,Pershing_RX_2020_LD01 (1),206.093386,Prescribed Fire,1264.020411,33.514535,-105.514613,1264.500292
53927,135028,2020,Listed Prescribed Fire Start Date(s): 2020-05-...,2_County Line Fuel Break (1),77.245054,Prescribed Fire,2258.237344,40.649556,-122.726618,2258.425715
53928,135047,2020,Listed Prescribed Fire Start Date(s): 2020-06-...,1_FillyII underburn (1),19.824395,Prescribed Fire,2080.377015,47.490303,-116.670445,2080.544867


## Save that data!

In [24]:
fire_distances.to_csv('fire_distances.csv', index=False)
